<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/NBmodel_on_parsed_AEARCT_toApplyOn3IESource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
# as of today
import datetime as dt
dt.date.today()
import os
import pandas as pd
from google.colab import drive
import csv
import os
data_dir = '/content/drive/MyDrive/finalMetaFile'
files = os.listdir(data_dir)

Mounted at /content/drive


# Train on AEA RCT paper abstract

In [31]:
df = pd.read_csv('/content/drive/MyDrive/finalMetaFile/AEA RCT trials with extracted info.csv')
df_aea=df[['Paper_Abstract','Paper_Article Title','Relevant papers for csv']].dropna()
import re
df_aea['title'] = df_aea['Relevant papers for csv'].str.extract(r'"(.*?)"', expand=False)
df_aea=df_aea.rename(columns={'Paper_Abstract': 'abstract'})
print(len(df_aea),df_aea['title'].isna().sum())

625 330


In [32]:
df_aea['label'] = 1

In [34]:
df_aea=df_aea.drop(columns=['Paper_Article Title','Relevant papers for csv'])
df_aea = df_aea[df_aea['title'].notnull()]
df_aea

,abstract,title,label
18,"In 2008, Oregon initiated a limited expansion ...",Medicaid Increases Emergency Department Use: E...,1
22,Low willingness to pay (WTP) for environmental...,Microcredit and willingness to pay for environ...,1
28,We designed a commitment savings product for a...,Tying Odysseus to the Mast: Evidence from a Co...,1
41,We study how career incentives affect who sele...,"Do-Gooders and Go-Getters: Career Incentives, ...",1
43,"Objectives\r\nPregnancy, labour, and delivery ...",‘Little Bit Afraid ‘Til I Found How It Was’: C...,1
...,...,...,...
5154,New technologies allow perfect detection of en...,Man vs. machine: Technological promise and pol...,1
5683,"Despite its empirical relevance, increasing re...",Sharing Rules in Bertrand Oligopolies with Inc...,1
5739,Under-refinancing limits the transmission of a...,"The Last Mile of Monetary Policy: Inattention,...",1
5943,In a time when politics seem to be polarized a...,Democratic Socialism,1


In [35]:
len(df_aea)

295

## Attached AEA to 3IE

In [36]:
df2 = pd.read_csv('/content/drive/MyDrive/finalMetaFile/3ie_parsed_metadata.csv')

# 1. Title based classfication

In [39]:

import pandas as pd
df2['label'] = df2['method'].apply(lambda x: 1 if pd.notnull(x) and "Randomised controlled trial" in x else 0)
df_ready=df2[['title','abstract','label']].append(df_aea)
df_ready.columns

<ipython-input-39-f3ab401072f0>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ready=df2[['title','abstract','label']].append(df_aea)


Index(['title', 'abstract', 'label'], dtype='object')

In [40]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [41]:
import string

def preprocess_text(text):
  text = text.lower()
  text = ''.join([c for c in text if c not in string.punctuation])
  #text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
  return text

data =  df_ready
data["Text"] = data["title"].apply(preprocess_text)


vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(data["Text"])

In [42]:
X_train, X_test, y_train, y_test = train_test_split(features, data["label"], test_size=0.2)

In [43]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.7370


# 2. Abstract based

In [44]:
data =  df_ready
data["Text"] = data["abstract"].fillna('NaN').apply(preprocess_text)

vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(data["Text"])
X_train, X_test, y_train, y_test = train_test_split(features, data["label"], test_size=0.2)
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7335
